In [1]:
!pip install faker pandas numpy



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [19]:
import pandas as pd
import numpy as np
from faker import Faker
import random

# Initialize the Faker generator
fake = Faker()

# Define parameters
num_users = 10000  # Number of users
num_exercises = 100  # Number of exercises

# Define exercises
exercises = ['Squats', 'Push-Ups', 'Lunges', 'Deadlifts', 'Bench Press', 
             'Shoulder Press', 'Bicep Curls', 'Tricep Dips', 'Leg Press', 
             'Plank']

# Create a DataFrame to hold the exercise data
data = []

for user in range(num_users):
    # Randomly select a sequence of exercises for each user
    exercise_sequence = random.choices(exercises, k=num_ratings_per_user)
    
    for exercise in exercise_sequence:
        # Randomly generate reps and sets
        reps = random.randint(5, 15)  # Random number of repetitions
        sets = random.randint(2, 5)  # Random number of sets
        
        data.append({
            'user': f'User_{user}',
            'exercise': exercise,
            'reps': reps,
            'sets': sets,
        })

# Create the DataFrame
df = pd.DataFrame(data)

# Save the dataset to a CSV file
df.to_csv('fake_gym_exercise_data_collab.csv', index=False)
print("Dataset saved as 'fake_gym_exercise_data_collab.csv'")


Dataset saved as 'fake_gym_exercise_data_collab.csv'


In [20]:
df.shape

(50000, 4)

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset
df = pd.read_csv('fake_gym_exercise_data_collab.csv')


# Create a user-item matrix to hold reps and sets for each exercise
exercise_matrix = df.pivot_table(index='exercise', values=['reps', 'sets'], aggfunc='mean').fillna(0)

# Function to get recommendations based on exercise name, reps, and sets
def get_exercise_recommendations(exercise_name, reps, sets, n_recommendations=3):
    # Calculate similarity based on reps and sets
    exercise_features = exercise_matrix.loc[exercise_name].values.reshape(1, -1)
    
    # Compute similarity between the input exercise and all others
    similarity_scores = cosine_similarity(exercise_features, exercise_matrix).flatten()
    
    # Create a DataFrame for exercises with their similarity scores
    exercise_similarity = pd.DataFrame({
        'exercise': exercise_matrix.index,
        'similarity': similarity_scores
    })
    
    # Filter out the exercise itself
    exercise_similarity = exercise_similarity[exercise_similarity['exercise'] != exercise_name]
    
    # Sort by similarity score
    recommended_exercises = exercise_similarity.sort_values(by='similarity', ascending=False)

    # Return the top N recommendations
    return recommended_exercises['exercise'].head(n_recommendations).tolist()

# Example usage
exercise_name = 'Squats'
reps = 10
sets = 3
recommended_exercises = get_exercise_recommendations(exercise_name, reps, sets, n_recommendations=3)
print(f"Recommended exercises based on '{exercise_name}' with {reps} reps and {sets} sets: {recommended_exercises}")


Recommended exercises based on 'Squats' with 10 reps and 3 sets: ['Bicep Curls', 'Bench Press', 'Deadlifts']
